In [6]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    SceneGraph,
    Simulator,
    StartMeshcat,
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState

In [5]:
### Run this only once (each instance opens a port for the visualizer)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [14]:
### Adapted from Underactuated textbook
# Build the acrobot diagram
builder = DiagramBuilder()
acrobot = builder.AddSystem(AcrobotPlant())  # TODO implement the dynamics myself?

scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.Set2dRenderMode(xmin=-6, xmax=4, ymin=-5, ymax=4)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

# Set up a simulator to run the diagram
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# Set initial conditions (state is theta1, theta2, theta1dot, theta2dot)
context.SetContinuousState([0, 0, 0.5, 0])

simulator.set_target_realtime_rate(1.0)
for _ in range(10):
    simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)